# **Job Data Web Search Project**

## **Section 1 - Design Data Warehouse**

### 1-1 Read in Glassdoor Job Dataset

In [1]:
import pandas as pd
df = pd.read_csv('glassdoor_jobs_with_index.csv',encoding="ISO-8859-1")

# get the column names from the dataframe
header = list(df.columns.values)
print(header)
df.head()
len(header)

['index', 'company_name', 'job_title', 'location', 'links', 'description']


6

### 1-2  Connect to local PostgreSQL

In [4]:
import psycopg2

# connect to PostgreSQL server
conn = psycopg2.connect(host='localhost', port='5432', dbname='flask_search', 
                        user='postgres', password='postgres')

# Create table name in postgresql
pgTable = "info"

### 1-3 Design the Table Schema

In [5]:
# get cursor object from db connection
cursor = conn.cursor()

# create table statement using same pandas header for column name
sqlCreateTable = "CREATE TABLE IF NOT EXISTS "+pgTable+" ("\
"{0} int PRIMARY KEY, "\
"{1} varchar(100),"\
"{2} varchar(500),"\
"{3} varchar(500),"\
"{4} varchar(10000),"\
"{5} varchar(10000));".format(*header) # unpack the list of header to format sql string
print(sqlCreateTable)

# execute table creation
cursor.execute(sqlCreateTable)
conn.commit()
cursor.close()

CREATE TABLE IF NOT EXISTS info (index int PRIMARY KEY, company_name varchar(100),job_title varchar(500),location varchar(500),links varchar(10000),description varchar(10000));


### 1-4 Insert the Job Data into PostgreSQL

In [6]:
import psycopg2.extras as extras

def pd2pg(conn, df, table):
    """
    Transform Pandas dataframe to postgres table
    """
    # convert dataframe values to tuple
    tuples = [tuple(x) for x in df.to_numpy()]    
    columns = ','.join(list(df.columns))
    # generate a list of placeholders to create query string
    placeholders = ",".join(['%s' for i in range(len(header))])
    # query string
    query  = "INSERT INTO %s(%s) VALUES (%s)" % (table, columns, placeholders)
    print(query)    
    cursor = conn.cursor()
    try:
        cursor.executemany(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("Finish data insertion")
    cursor.close()

# execute data insert
pd2pg(conn, df, pgTable)

INSERT INTO info(index,company_name,job_title,location,links,description) VALUES (%s,%s,%s,%s,%s,%s)
Finish data insertion
